<a href="https://colab.research.google.com/github/G-Shillcock/Division_of_Labour/blob/main/DoLSymbolicBigGroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This section determines the conditions under which the division of labor is favored, taking into account task difficulties, group size, and assortativity.

\begin{align*}
W &= (1-λ)\left(\sum_i^n A_iB_i\right) + λ \left(\sum_{i\neq j}A_iB_j\right)\\
&=(1-2λ)\left(\sum_i^n A_iB_i\right) + λ \left(\sum_{i}A_i\right)\left(\sum_j B_j\right)\\
A_i &= (t_i)^a\\
B_i &= (1-t_i)^b
\end{align*}

In [5]:
from sympy import symbols, Eq, IndexedBase, Sum, solve, KroneckerDelta

t_A = symbols('t_A')                      # undivided ancestoral allocation
i, j = symbols('i, j', integer=True)        # group member indices
λ = symbols('λ')                            # shareability
a, b = symbols('a, b', positive=True)       # task elasticities
c = symbols('c', positive=True)             # workaround so that n >=2
n = c + 2                                   # group size

t = IndexedBase('t')                        # allocations
A = IndexedBase('A')                        # performances in task A
B = IndexedBase('B')                        # performances in task B

W = (Sum(λ*A[i]*B[j],(i,0,n-1),(j,0,n-1)) + Sum((1-2*λ)*A[i]*B[i],(i,0,n-1))).doit()

# W = (Sum(λ*A[i]*B[j]/(n**2),(i,0,n-1),(j,0,n-1)) + Sum((1-2*λ)*A[i]*B[i]/n,(i,0,n-1))).doit()
# equivalent to n=1 case.

# W = W /( n*(λ*(n-2)+1) )
# has no effect

W = W.replace(A[i], t[i]**a).replace(B[j],(1-t[j])**b).replace(B[i],(1-t[i])**b).doit()

W_A = W.replace(t[i],t_A).replace(t[j],t_A).doit()

display(Eq(t_A,solve(W_A.diff(t_A),t_A)[0]))

Sum((1 - 2*λ)*A[i]*B[i], (i, 0, c + 1)) + Sum(λ*A[i]*B[j], (i, 0, c + 1), (j, 0, c + 1))

Eq(t_A, a/(a + b))

The allocation of the undivided ancestoral population is equal to the relative elasticity of the tasks. We can use this value to find when division of labour yeilds a fitness advantage.

In [6]:
W00 = W.diff(t[0],t[0]).replace(t[i],t_A).replace(t[j],t_A).simplify()
W10 = W.diff(t[1],t[0]).replace(t[i],t_A).replace(t[j],t_A).simplify()
W01 = W.diff(t[0],t[1]).replace(t[i],t_A).replace(t[j],t_A).simplify()
W11 = W.diff(t[1],t[1]).replace(t[i],t_A).replace(t[j],t_A).simplify()

concavity = (W00+W11-W01-W10).subs(t_A, a/(a+b)).simplify()

sol = solve(concavity,b)[0]

display(Eq(b,sol))

Eq(b, a*(c*λ + 1)/(2*a*c*λ + 4*a*λ - c*λ - 1))

This can be rearranged to seperate parameters relating to the tasks from parameters related to the group's structure. Recall that $c = n + 2$.

In [7]:
display(Eq(1/a+1/b,(1/sol+1/a).simplify()))

Eq(1/b + 1/a, 2*λ*(c + 2)/(c*λ + 1))

---
\
Lets now double check that division of labour is only possible when intermediate resources are shared.

In [15]:
W_c= W.subs(λ,0).doit()

W00 = W_c.diff(t[0],t[0]).replace(t[i],t_A).replace(t[j],t_A).simplify()
W10 = W_c.diff(t[1],t[0]).replace(t[i],t_A).replace(t[j],t_A).simplify()
W01 = W_c.diff(t[0],t[1]).replace(t[i],t_A).replace(t[j],t_A).simplify()
W11 = W_c.diff(t[1],t[1]).replace(t[i],t_A).replace(t[j],t_A).simplify()

concavity = (W00+W11-W01-W10).subs(t_A, a/(a+b)).simplify()

solve(concavity,b)
# the solver returns no solution because the concavity is never zero.

[]

If we consider the case $\lambda=1/2$ and add Cobb parameters instead.

In [42]:
α, β = symbols('α, β', positive=True)
d = symbols('d', positive=True)

W_t = (Sum(A[i],(i,0,n-1))**α*Sum(B[i],(i,0,n-1))**β).doit()

W_t = W_t.replace(A[i], t[i]**a).replace(B[i],(1-t[i])**b).doit()

W_tA = W_t.replace(t[i],t_A).doit()
display(Eq(t_A,solve(W_tA.diff(t_A),t_A)[0]))

W00 = W_t.diff(t[0],t[0]).replace(t[i],t_A).simplify()
W10 = W_t.diff(t[1],t[0]).replace(t[i],t_A).simplify()
W01 = W_t.diff(t[0],t[1]).replace(t[i],t_A).simplify()
W11 = W_t.diff(t[1],t[1]).replace(t[i],t_A).simplify()

concavity = (W00+W11-W01-W10).subs(t_A, a*α/(a*α+b*β)).simplify()

# display(solve(concavity,b)[0])

solve(concavity,b)[0].replace(β,(1-d)).replace(α,d).simplify()

Eq(t_A, a*α/(a*α + b*β))

a*d/(a + d - 1)